In [1]:
!pip install node2vec tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 462 kB 5.9 MB/s 
  Created wheel for node2vec: filename=node2vec-0.4.3-py3-none-any.whl size=5980 sha256=3182c01af2b4a79b80fc1c9dd8257c5fd9d48b97199bc6d26fff92f619f86a87
  Stored in directory: /root/.cache/pip/wheels/07/62/78/5202cb8c03cbf1593b48a8a442fca8ceec2a8c80e22318bae9
Successfully built node2vec


In [ ]:
# optionally load copy
!7z x copy.zip -o/content


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1217300 bytes (1189 KiB)

Extracting archive: copy.zip
--
Path = copy.zip
Type = zip
Physical Size = 1217300

  0%    
Would you like to replace the existing file:
  Path:     /content/.config/.feature_flags_config.yaml
  Size:     101 bytes (1 KiB)
  Modified: 2022-05-03 13:41:39
with the file from archive:
  Path:     content/.config/.feature_flags_config.yaml
  Size:     101 bytes (1 KiB)
  Modified: 2022-05-03 13:41:39
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? A

  0% 2 - content/.config/.feature_flags_config.yaml                                                   Everything is 

**-------------- Functions --------------**

In [77]:
# choose links from AS file
# samples_number for each link type (peer/customer)
def get_as_data(as_relation_filename, samples_number):
  as_relation_file = open(as_relation_filename, 'r')

  peers = 0
  customers = 0

  X = []
  Y = []

  for line in as_relation_file:
      if line.startswith('#'):
          continue
      data = line.split('|')

      if '0' in data[2]:
          if peers < samples_number:
            X.append([data[0], data[1]])
            Y.append(data[2])
            peers += 1
      else:
          if customers < samples_number:
            X.append([data[0], data[1]])
            Y.append(data[2])
            customers += 1
  return X, Y


def get_as_data2(as_relation_filename, samples_number):
  as_relation_file = open(as_relation_filename, 'r')

  peers = 0

  X = []
  Y = []

  for line in as_relation_file:
      if line.startswith('#'):
          continue
      data = line.split('|')

      if peers < samples_number:
        X.append([data[0], data[1]])
        Y.append(data[2])
        peers += 1
       
  return X, Y

def get_as_data3(as_relation_filename, samples_number, skip):
  as_relation_file = open(as_relation_filename, 'r')

  peers = 0
  skip_ = 0

  X = []
  Y = []

  for line in as_relation_file:
      if line.startswith('#'):
          continue

      if skip_ < skip:
        skip_ += 1
        continue
        
      data = line.split('|')

      if peers < samples_number:
        X.append([data[0], data[1]])
        Y.append(data[2])
        peers += 1
       
  return X, Y



import csv

# create dataset for neural network in format:
# [start_node_embedding], [end_node_embedding], [link_type]
def create_dataset(X, Y, X_embed, dataset_filename):

  out = open(dataset_filename, 'w')
  writer = csv.writer(out)

  for i in range(len(X)):
      data = []
      data.extend(X_embed[X[i][0]])
      data.extend(X_embed[X[i][1]])
      if '0' in Y[i]:
          data.append('0')
      else:
          data.append('1')
      
      writer.writerow(data)

  out.close()


# count links
def display_link_stats(Y):

  peers = 0
  customers = 0

  for y in Y:
      if int(y) == 0:
          peers = peers + 1
      else:
          customers = customers + 1

  print('Peer:', peers, '\n  Customer:', customers)




# embeddings methods ----------------------------------

import networkx as nx
from node2vec import Node2Vec

def node2vec_get_embeddings(as_data):

  embedding_filename = "embeddings_tmp_file"

  # Create a graph 
  graph = nx.Graph()
  for AS in as_data:
      graph.add_edge(AS[0], AS[1])

  # Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
  node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4)  # Use temp_folder for big graphs

  # Embed nodes
  model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `dimensions` and `workers` are automatically passed (from the Node2Vec constructor)

  # Save embeddings for later use
  model.wv.save_word2vec_format(embedding_filename)

  embeddings_file = open(embedding_filename,'r')
  node_embeddings = {}

  i = 0
  for line in embeddings_file:
      if i == 0:
          i = i + 1
          continue
      data = line.split(' ')
      node_embeddings[data[0]] = data[1:]

  return node_embeddings
  


**-------------- Create datasets --------------**

In [6]:
#create embeddings and basic dataset
samples = 2000

X, Y = get_as_data('as-rank.caida.peercones-with-IX.txt', samples / 2) # balanced
display_link_stats(Y)

X_embed = node2vec_get_embeddings(X)
create_dataset(X, Y, X_embed, 'dataset_basic.csv')


Peer: 558 
  Customer: 1000


Computing transition probabilities:   0%|          | 0/1238 [00:00<?, ?it/s]

In [82]:
# create datasets for training and testing
import csv

def split_data(X, Y):
  
  X_0 = []
  Y_0 = []

  X_1 = []
  Y_1 = []

  for i in range(len(X)):
      if Y[i][0] == '0':
        X_0.append(X[i])
        Y_0.append([0])
      else:
        X_1.append(X[i])
        Y_1.append([1])
        
  return X_0, Y_0, X_1, Y_1

def write_csv(filename, X, Y):
  out = open(filename, 'w')
  writer = csv.writer(out)
  for i in range(len(X)):
    data = []
    data.extend(X[i])
    data.extend(Y[i])
    writer.writerow(data)


X = []
Y = []

with open('dataset_basic.csv', mode='r') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  for row in csv_reader:
    X.append(row[0:128])
    Y.append(row[128:])
    

X_0, Y_0, X_1, Y_1 = split_data(X, Y)

X.extend(X_0[0:500])
X.extend(X_1[0:500])

Y.extend(Y_0[0:500])
Y.extend(Y_1[0:500])

write_csv('dataset_balanced.csv', X, Y)

X.clear()
Y.clear()
X.extend(X_0[500:550])
X.extend(X_1[500:550])

Y.extend(Y_0[500:550])
Y.extend(Y_1[500:550])


write_csv('test_dataset_balanced.csv', X, Y)

X.clear()
Y.clear()

X.extend(X_1[550:650])
Y.extend(Y_1[550:650])

write_csv('test_dataset_unbalanced_one.csv', X, Y)


X.clear()
Y.clear()

X.extend(X_0[500:550])
Y.extend(Y_0[500:550])

write_csv('test_dataset_unbalanced_zero.csv', X, Y)


**---- Train and evaluate model ----**

In [93]:
# create model of neural network

def create_model():
  model = Sequential()
  model.add(Dense(128, input_dim=128, activation='relu'))
  model.add(Dense(64, input_dim=128, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [89]:
# create, train and save model
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

# load dataset
dataframe = read_csv("dataset_balanced.csv", header=None)
# dataframe = shuffle(dataframe)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:128].astype(float)
Y = dataset[:,128].astype(int)

# evaluate model with standardized dataset
n_split=3
k = 1

for train_index,test_index in KFold(n_splits=n_split, shuffle=True).split(X):
	x_train,x_test=X[train_index],X[test_index]
	y_train,y_test=Y[train_index],Y[test_index]
	validation_data = [x_test, y_test];

	print("K =", k);
	k += 1

	model=create_model()
	model.fit(x_train, y_train, epochs=20, batch_size=25, validation_data=validation_data, verbose=2)


model.save("model")

K = 1
Epoch 1/20
69/69 - 1s - loss: 0.2896 - accuracy: 0.9290 - val_loss: 0.1001 - val_accuracy: 0.9719 - 965ms/epoch - 14ms/step
Epoch 2/20
69/69 - 0s - loss: 0.0617 - accuracy: 0.9853 - val_loss: 0.0387 - val_accuracy: 0.9894 - 199ms/epoch - 3ms/step
Epoch 3/20
69/69 - 0s - loss: 0.0310 - accuracy: 0.9918 - val_loss: 0.0246 - val_accuracy: 0.9953 - 206ms/epoch - 3ms/step
Epoch 4/20
69/69 - 0s - loss: 0.0161 - accuracy: 0.9965 - val_loss: 0.0095 - val_accuracy: 0.9965 - 187ms/epoch - 3ms/step
Epoch 5/20
69/69 - 0s - loss: 0.0171 - accuracy: 0.9965 - val_loss: 0.0350 - val_accuracy: 0.9965 - 207ms/epoch - 3ms/step
Epoch 6/20
69/69 - 0s - loss: 0.0145 - accuracy: 0.9977 - val_loss: 0.0077 - val_accuracy: 0.9965 - 184ms/epoch - 3ms/step
Epoch 7/20
69/69 - 0s - loss: 0.0117 - accuracy: 0.9971 - val_loss: 0.0268 - val_accuracy: 0.9965 - 222ms/epoch - 3ms/step
Epoch 8/20
69/69 - 0s - loss: 0.0131 - accuracy: 0.9971 - val_loss: 0.0110 - val_accuracy: 0.9965 - 183ms/epoch - 3ms/step
Epoch 9/2

In [92]:
# evaluate loaded model
from tensorflow import keras
from pandas import read_csv
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

import numpy as np

# load dataset
dataframe = read_csv("test_dataset_balanced.csv", header=None)
#dataframe = read_csv("test_dataset_unbalanced_zero.csv", header=None)
#dataframe = read_csv("test_dataset_unbalanced_one.csv", header=None)

dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:128].astype(float)
Y = dataset[:,128]

display_link_stats(Y)

model = keras.models.load_model('model')
predictions = (model.predict(X) > 0.5).astype(int)

success = 0
for i in range(len(X)):
  #print(Y[i], " -> ", predictions[i])
  if int(predictions[i]) == int(Y[i]):
    success += 1

print("Result :", success/len(X)*100)


Peer: 50 
  Customer: 50
Result : 100.0


In [94]:
!rm -rf sample_data
!7z a copy.zip /content/*


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive:
  0M Scan  /content/                    3 folders, 12 files, 8937828 bytes (8729 KiB)

Creating archive: copy.zip

Items to compress: 15

  0%     18% 2 + dataset_balanced.csv                              40% 3 + dataset_basic.csv                           62% 3 + dataset_basic.csv                           82% 4 + embeddings_tmp_file                            
Files read from disk: 12
Archive size: 2254683 bytes (2202 KiB)
Everything is Ok
